In [1]:
import os
import torch
import torch.nn.functional as F
import torchmetrics
import math
import json

from main import get_model
from dataset.dataset import (
    LoaderConstructor,
    create_poetryfoundation_dataset,
    create_story_txt_dataset
)

In [2]:
cfg = [
   {
        "dataset": "poetryfoundation",
        "batch_size": 3,
        "max_length": 20,
        "embed_dim": 512,
        "min_text_length": 50,
    },
    {
        "dataset": "story_txt",
        "batch_size": 3,
        "max_length": 20,
        "embed_dim": 512,
        "min_text_length": 50,
    }
]

def load_and_construct(cfg):
    print(f"\nEvaluating dataset: {cfg['dataset']}")
    
    # Load the dataset
    if cfg["dataset"] == "poetryfoundation":
        dataset = create_poetryfoundation_dataset(os.getcwd())
    elif cfg["dataset"] == "story_txt":
        dataset = create_story_txt_dataset(os.getcwd())
    else:
        raise ValueError(f"Unknown dataset: {cfg['dataset']}")

    # Construct the dataloaders
    lc = LoaderConstructor(
        dataset=dataset,
        batch_size=cfg["batch_size"],
        max_length=cfg["max_length"],
        labels_sequence=False,
        min_freq=3,
    )

    loaders = {loader: lc.construct_loader(split=loader) for loader in ["train", "validation", "test"]}

    input_size = getattr(loaders["train"].dataset, "input_size", None)
    if input_size is None:
        raise AttributeError("Dataset object does not have 'input_size' attribute. Check implementation.")

    vocab_size = lc.vocab_size
    output_size = lc.output_size
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    
    return dataset, lc, loaders, input_size, vocab_size, output_size, device


In [3]:
def get_best_model_weights(dataset):
    """Finds the best model weight files for both LSTM and Transformer based on the highest accuracy."""
    model_types = ["lstm", "transformer"]
    best_models = {}
    
    for model_name in model_types:
        model_files = [
            f"trained_models/{model_name}_{dataset}_lr=0_001_best.pt",
            f"trained_models/{model_name}_{dataset}_lr=0_0001_best.pt",
            f"trained_models/{model_name}_{dataset}_lr=0_0005_best.pt"
        ]
        
        best_model = None
        best_accuracy = -1
        
        for model_file in model_files:
            stats_file = model_file.replace("_best.pt", "_stats.json")
            if os.path.exists(stats_file):
                with open(stats_file, "r") as f:
                    stats = json.load(f)
                    
                    for lr, details in stats.items():
                        if "val_accuracy" in details:
                            accuracy = details["val_accuracy"]
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_model = details["best_model"]
        
        if best_model:
            best_models[model_name] = best_model

    return best_models

In [4]:
model_weights = {
    "poetryfoundation": get_best_model_weights("poetryfoundation"),
    "story_txt": get_best_model_weights("story_txt")
}

def initialize_model(model_name, vocab_size, input_size, output_size, device, cfg):
    """Loads the model with trained weights"""
    model = get_model(
        model=model_name,
        vocab_size=vocab_size,
        embed_dim=cfg["embed_dim"],
        seq_len=input_size,
        output_dim=output_size,
        device=device,
    )
    best_model_weight = model_weights[cfg['dataset']][model_name]
    
    model.load_state_dict(torch.load(best_model_weight))
    model.to(device).eval()
    return model


In [ ]:
def calculate_perplexity(loss):
    return math.exp(loss)

def nucleus_sampling(probs, top_p=0.8, min_top_k=5):
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

    top_indices = sorted_indices[cumulative_probs <= top_p]
    if len(top_indices) < min_top_k:
        top_indices = sorted_indices[:min_top_k]

    return top_indices[torch.randint(len(top_indices), (1,))].item()


def evaluate(model, batch, output_size, device, lc, test=False, n_next_words=5):
    criterion = torch.nn.CrossEntropyLoss()
    accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=output_size).to(device)
    top3_accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=output_size, top_k=5).to(device)
    
    """Evaluates the model and predicts next words for poetry"""
    inputs, labels = batch["input_ids"].to(device), batch["labels"].to(device)
    labels = labels.contiguous().view(-1)
    output = model(inputs).view(-1, output_size)

    loss = criterion(output, labels).item()
    accuracy = accuracy_metric(torch.argmax(output, dim=-1), labels).item()
    perplexity = calculate_perplexity(loss)
    top3_accuracy = top3_accuracy_metric(output, labels).item()

    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Perplexity: {perplexity:.4f}")
    print(f"Top-3 Accuracy: {top3_accuracy:.4f}")

    # Predict next words for new poetic lines
    if test:
        mini_dataset = [
            {"text": "The moonlight danced upon the silent lake", "predictions": []},
            {"text": "A whisper of wind through the autumn leaves", "predictions": []},
            {"text": "The poet's heart was filled with endless wonder", "predictions": []},
            {"text": "Beneath the stars, she walked alone", "predictions": []},
            {"text": "His voice was music, soft and deep", "predictions": []},
        ]

        # Tokenize the texts
        tokenised_samples = lc.tokenizer.create_tokens(mini_dataset)
        tokenised_samples = lc.tokenizer.pad_sequences(tokenised_samples)
        encodings = lc.tokenizer.encode(tokenised_samples)

        # No labels in this case
        inputs = encodings[:, 1:].to(device)
        decoded_inputs = [lc.tokenizer.decode(sample.tolist(), target=False) for sample in inputs]

        with torch.no_grad():
            for _ in range(n_next_words):
                output = model(inputs)
                temperature = 0.8
                probs = torch.softmax(output[:, -1, :]/temperature, dim=-1) if output.dim() == 3 else torch.softmax(output, dim=-1)

                next_tokens = []

                for i in range(len(mini_dataset)):
                    next_token = nucleus_sampling(probs[i], top_p=0.8) # Top-p (nucleus) sampling
                    next_tokens.append(next_token)
                decoded_tokens = lc.tokenizer.decode(next_tokens, target=True)

                # Encode the tokens to get the input for the next iteration
                encoded_tokens = lc.tokenizer.encode([decoded_tokens]).to(device)

                # Concatenate the input with the new tokens
                inputs = torch.cat([inputs[:, 1:], encoded_tokens.reshape(-1, 1)], dim=1)

                for i in range(len(mini_dataset)):
                    mini_dataset[i]["predictions"].append(decoded_tokens[i])

        for sample, input_sentence in zip(mini_dataset, decoded_inputs):
            print(f"Input: {' '.join(input_sentence)}")
            print(f"Predicted next words: {' '.join(map(str, sample['predictions']))}\n")
            print()
    else:      
        for i in range(batch["input_ids"].shape[0]):
            input_text = lc.tokenizer.decode(inputs[i].tolist(), target=False)
            target_text = lc.tokenizer.decode(labels[i].unsqueeze(0).tolist(), target=True)[0]
            predicted_text = lc.tokenizer.decode(torch.argmax(output[i], dim=-1).unsqueeze(0).tolist(), target=True)[0]

            print(f"Input: {' '.join(input_text)}")
            print(f"Target: {target_text}")
            print(f"Predicted: {predicted_text}")
            print()


In [478]:
# **Run Evaluation**
dataset, lc, loaders, input_size, vocab_size, output_size, device = load_and_construct(cfg[0])
batch = next(iter(loaders["train"]))
for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name, vocab_size, input_size, output_size, device, cfg[0])
    evaluate(model, batch, output_size, device, lc, test=False)
    print("*" * 50)



Evaluating dataset: poetryfoundation
**************************************************
Evaluating model: lstm
**************************************************
Loss: 4.1628
Accuracy: 0.3333
Perplexity: 64.2485
Top-3 Accuracy: 0.6667
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 113 complaint of the absence of her love being upon the
Target: sea
Predicted: world

Input: poem “breadth . circle . desert . monarch . month . wisdom . ( for which there are no rhymes
Target: )
Predicted: )

Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 159 balcony
Target: scene
Predicted: song

**************************************************
**************************************************
Evaluating model: transformer
**************************************************
Loss: 3.5527
Accuracy: 0.3333
Perplexity: 34.9087
Top-3 Accuracy: 1.0000
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 113 complaint of t

In [479]:
batch = next(iter(loaders["test"]))
for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name, vocab_size, input_size, output_size, device, cfg[0])
    evaluate(model, batch, output_size, device, lc, test=True, n_next_words=3)
    print("*" * 50)

**************************************************
Evaluating model: lstm
**************************************************
Loss: 6.2324
Accuracy: 0.0000
Perplexity: 508.9517
Top-3 Accuracy: 0.0000
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> the moonlight <oov> upon the silent lake
Predicted next words: novel art night


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> a <oov> of wind through the autumn leaves
Predicted next words: 51 character dance


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> the poet ' s heart was filled with endless wonder
Predicted next words: summer letter '


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> beneath the stars , she walked alone
Predicted next words: again down on


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> his voice was music , soft and deep
Predicted next words: ago somewhere rom

In [472]:
# **Run Evaluation** for story dataset
dataset, lc, loaders, input_size, vocab_size, output_size, device = load_and_construct(cfg[1])
batch = next(iter(loaders["train"]))

for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name, vocab_size, input_size, output_size, device, cfg[1])
    evaluate(model, batch, output_size, device, lc, test=False)
    print("*" * 50)


Evaluating dataset: story_txt
**************************************************
Evaluating model: lstm
**************************************************
Loss: 4.7881
Accuracy: 0.3333
Perplexity: 120.0766
Top-3 Accuracy: 0.3333
Input: represented as plain text streams , interspersing plain text data with sequences of characters that represent the additional data structures
Target: .
Predicted: .

Input: <pad> <pad> rachel hey mon , look what i just found on the floor . ( monica smiles . )
Target: what
Predicted: .

Input: . okay , i got one . do you remember that vegetarian pate that i made that you loved so
Target: much
Predicted: .

**************************************************
**************************************************
Evaluating model: transformer
**************************************************
Loss: 1.2142
Accuracy: 1.0000
Perplexity: 3.3676
Top-3 Accuracy: 1.0000
Input: represented as plain text streams , interspersing plain text data with sequences of character

In [477]:
batch = next(iter(loaders["test"]))
for model_name in ["lstm", "transformer"]:
    print("*" * 50)
    print(f"Evaluating model: {model_name}")
    print("*" * 50)
    model = initialize_model(model_name, vocab_size, input_size, output_size, device, cfg[1])
    evaluate(model, batch, output_size, device, lc, test=True, n_next_words=3)
    print("*" * 50)

**************************************************
Evaluating model: lstm
**************************************************
Loss: 2.5594
Accuracy: 0.6667
Perplexity: 12.9285
Top-3 Accuracy: 0.6667
Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> the moonlight danced upon the silent lake
Predicted next words: readers markup wildlife


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> a whisper of wind through the <oov> leaves
Predicted next words: windows jungle landscape


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> the <oov> ' s heart was filled with endless wonder
Predicted next words: together lot lot


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> beneath the stars , she walked alone
Predicted next words: nation together generations


Input: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> his voice was music , soft and deep
Predicte